In [1]:
from pyspark.sql import SparkSession
from pyspark.ml.recommendation import ALS
from pyspark.ml.evaluation import RegressionEvaluator

In [2]:
spark = SparkSession.builder.appName('rec').getOrCreate()

In [3]:
data = spark.read.csv('movielens_ratings.csv', inferSchema=True, header=True)

In [4]:
data.show()

+-------+------+------+
|movieId|rating|userId|
+-------+------+------+
|      2|   3.0|     0|
|      3|   1.0|     0|
|      5|   2.0|     0|
|      9|   4.0|     0|
|     11|   1.0|     0|
|     12|   2.0|     0|
|     15|   1.0|     0|
|     17|   1.0|     0|
|     19|   1.0|     0|
|     21|   1.0|     0|
|     23|   1.0|     0|
|     26|   3.0|     0|
|     27|   1.0|     0|
|     28|   1.0|     0|
|     29|   1.0|     0|
|     30|   1.0|     0|
|     31|   1.0|     0|
|     34|   1.0|     0|
|     37|   1.0|     0|
|     41|   2.0|     0|
+-------+------+------+
only showing top 20 rows



In [5]:
data.describe().show()

+-------+------------------+------------------+------------------+
|summary|           movieId|            rating|            userId|
+-------+------------------+------------------+------------------+
|  count|              1501|              1501|              1501|
|   mean| 49.40572951365756|1.7741505662891406|14.383744170552964|
| stddev|28.937034065088994| 1.187276166124803| 8.591040424293272|
|    min|                 0|               1.0|                 0|
|    max|                99|               5.0|                29|
+-------+------------------+------------------+------------------+



In [6]:
training, test = data.randomSplit([0.8,0.2])

In [7]:
als = ALS(maxIter=5, regParam=0.01, userCol='userId', itemCol='movieId', ratingCol='rating')

In [8]:
model = als.fit(training)

In [9]:
predictions = model.transform(test)

In [10]:
predictions.show()

+-------+------+------+-----------+
|movieId|rating|userId| prediction|
+-------+------+------+-----------+
|     31|   1.0|     4|  1.5072833|
|     31|   2.0|    25|   3.236711|
|     31|   1.0|    29|  1.4394377|
|     85|   1.0|    13|  1.9976081|
|     85|   5.0|     8|  4.3350987|
|     85|   1.0|    29| 0.81301045|
|     65|   1.0|    16| 0.66070396|
|     65|   2.0|     3|0.109331444|
|     65|   2.0|     5|-0.09411584|
|     65|   2.0|    15|  0.7980082|
|     65|   5.0|    23| 0.85331863|
|     53|   3.0|    20| 0.64993775|
|     53|   2.0|    19|   2.595832|
|     78|   1.0|    27|   1.029559|
|     78|   1.0|     1|  0.7655894|
|     78|   1.0|    13|  0.9668139|
|     34|   1.0|    17|  2.6983829|
|     34|   1.0|     4|  1.9002161|
|     34|   1.0|    14|  1.5592948|
|     81|   3.0|    26|  2.4412332|
+-------+------+------+-----------+
only showing top 20 rows



In [13]:
evaluator = RegressionEvaluator(metricName='rmse', labelCol='rating', predictionCol='prediction')

In [14]:
rmse = evaluator.evaluate(predictions)

In [15]:
print('RMSE')
print(rmse)

RMSE
1.7701131220042385


In [16]:
single_user = test.filter(test['userId'] == 11).select(['movieId','userId'])

In [17]:
single_user.show()

+-------+------+
|movieId|userId|
+-------+------+
|     10|    11|
|     16|    11|
|     64|    11|
|     70|    11|
|     77|    11|
|     79|    11|
|     89|    11|
|     99|    11|
+-------+------+



In [18]:
recommendations = model.transform(single_user)

In [19]:
recommendations.orderBy('prediction',ascending=False).show()

+-------+------+----------+
|movieId|userId|prediction|
+-------+------+----------+
|     10|    11| 3.5912805|
|     64|    11| 3.0039861|
|     89|    11| 2.6067276|
|     99|    11| 1.5814935|
|     70|    11|   0.77468|
|     79|    11|0.74428976|
|     77|    11| 0.7066345|
|     16|    11|  0.508194|
+-------+------+----------+

